In [25]:
import yaml
import os
import pandas as pd
from neo4j import GraphDatabase
from pathlib import Path
import shutil

In [26]:
with open('config.yaml', 'r') as file:
    PARAM = yaml.safe_load(file)


nodes:

habitat
country - continent
habitat type - hierarchy
service

ESVD2_0_Biome_Codes is a semicolon delimiter field. It contains the numerical code for ESVD2_0_Biome_1, ESVD2_0_Biome_2 and ESVD2_0_Biome_3 and so on. It corresponds to ESVD2_0_Biome. See https://www.esvd.info/_files/ugd/53b4f9_6bd1590ca0274bd0bd28be7865280900.pdf


TEEB – ECOSYSTEM SERVICES CLASSIFICATION, TEEB_ES is the upper category and TEEB_SubES is the subcategory under TEEB_ES
https://www.esvd.info/_files/ugd/53b4f9_1e5e0ba457744f3ea3e8f3d1bab78e88.pdf

CICES - COMMON INTERNATION CLASSIFICATION OF ECOSYSTEM SERVICES

SEEA - SYSTEM OF ECONOMIC-ENVIRONMENTAL ACCOUNTING, Page 131, Table 6.3 in https://seea.un.org/sites/seea.un.org/files/documents/EA/seea_ea_white_cover_final.pdf. ;-delimiter field

Countries and region csv from https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/blob/master/all/all.csv

Valuation Methods: https://www.es-partnership.org/wp-content/uploads/2020/08/ESVD_Global-Update-FINAL-Report-June-2020.pdf Page 9

In [27]:
cices = pd.read_csv('CICES.tsv', sep='\t')
teeb = pd.read_csv('TEEB.tsv', sep='\t')
biome = pd.read_csv('ESVD2_0_Biome.tsv', sep='\t')
countries = pd.read_csv('countries.csv')


data_df = pd.read_csv('Esvd_Full_Data_8th-Dec-2023_10-11-46_Database_Version_MAY2023V1.1.csv')

/tmp/ipykernel_11208/3109633001.py:7: DtypeWarning: Columns (7,8,9,10,11,20,21,22,23,24,25,26,35,53,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv('Esvd_Full_Data_8th-Dec-2023_10-11-46_Database_Version_MAY2023V1.1.csv')


In [28]:
for_neo4j_folder = "for_neo4j"

In [29]:
cices_section = set()
cices_division = set()
cices_group = set()
# cices_class = set()
# cices_class_type = set()
cices_class = {}

relation_cices_section_division = set()
relation_cices_division_group = set()
relation_cices_group_class = set()


for index, row in cices.iterrows():
    section = row['section']
    division = row['division']
    code = row['code']
    group = row['group'] + f" ({code[0]})"
    class_ = row['class']
    class_type = row['class_type']
    

    cices_section.add(section)
    cices_division.add(division)
    cices_group.add(group)
    # cices_class_type.add(class_type)
    cices_class[code] = {"class_type": class_type, "class_": class_}

    relation_cices_section_division.add((section, division))
    relation_cices_division_group.add((division, group))
    relation_cices_group_class.add((group, code))


with open(os.path.join(for_neo4j_folder, 'node_cices_section.tsv'), 'w') as file:
    file.write('name\n')
    for section in cices_section:
        file.write(f'{section}\n')

with open(os.path.join(for_neo4j_folder, 'node_cices_division.tsv'), 'w') as file:
    file.write('name\n')
    for division in cices_division:
        file.write(f'{division}\n')

with open(os.path.join(for_neo4j_folder, 'node_cices_group.tsv'), 'w') as file:
    file.write('name\n')
    for group in cices_group:
        file.write(f'{group}\n')


with open(os.path.join(for_neo4j_folder, 'node_cices_class.tsv'), 'w') as file:
    file.write('code\tclass_type\tname\n')
    for code in cices_class:
        file.write(f'{code}\t{cices_class[code]["class_type"]}\t{cices_class[code]["class_"]}\n')

with open(os.path.join(for_neo4j_folder, 'relation_cices_section_division.tsv'), 'w') as file:
    file.write('from\tto\n')
    for section, division in relation_cices_section_division:
        file.write(f'{division}\t{section}\n')

with open(os.path.join(for_neo4j_folder, 'relation_cices_division_group.tsv'), 'w') as file:
    file.write('from\tto\n')
    for division, group in relation_cices_division_group:
        file.write(f'{group}\t{division}\n')

with open(os.path.join(for_neo4j_folder, 'relation_cices_group_class.tsv'), 'w') as file:
    file.write('from\tto\n')
    for group, code in relation_cices_group_class:
        file.write(f'{code}\t{group}\n')



In [30]:
teeb_category = set()
teeb_ecosystem_service = set()
teeb_ecosystem_subservice = {}

relation_teeb_category_ecosystem_service = set()
relation_teeb_ecosystem_service_ecosystem_subservice = set()

for index, row in teeb.iterrows():
    category = row['category']
    ecosystem_service = row['ecosystem_service']
    ecosystem_subservice = row['ecosystem_sub_service']
    code = row['code']

    teeb_category.add(category)
    teeb_ecosystem_service.add(ecosystem_service)
    teeb_ecosystem_subservice[code] = ecosystem_subservice

    relation_teeb_category_ecosystem_service.add((category, ecosystem_service))
    relation_teeb_ecosystem_service_ecosystem_subservice.add((ecosystem_service, code))

with open(os.path.join(for_neo4j_folder, 'node_teeb_category.tsv'), 'w') as file:
    file.write('name\n')
    for category in teeb_category:
        file.write(f'{category}\n')

with open(os.path.join(for_neo4j_folder, 'node_teeb_ecosystem_service.tsv'), 'w') as file:
    file.write('name\n')
    for ecosystem_service in teeb_ecosystem_service:
        file.write(f'{ecosystem_service}\n')

with open(os.path.join(for_neo4j_folder, 'node_teeb_ecosystem_subservice.tsv'), 'w') as file:
    file.write('code\tname\n')
    for code in teeb_ecosystem_subservice:
        file.write(f'{code}\t{teeb_ecosystem_subservice[code]}\n')

with open(os.path.join(for_neo4j_folder, 'relation_teeb_category_ecosystem_service.tsv'), 'w') as file:
    file.write('from\tto\n')
    for category, ecosystem_service in relation_teeb_category_ecosystem_service:
        file.write(f'{ecosystem_service}\t{category}\n')

with open(os.path.join(for_neo4j_folder, 'relation_teeb_ecosystem_service_ecosystem_subservice.tsv'), 'w') as file:
    file.write('from\tto\n')
    for ecosystem_service, code in relation_teeb_ecosystem_service_ecosystem_subservice:
        file.write(f'{code}\t{ecosystem_service}\n')

In [31]:

biome_biome = {}
biome_ecozone = {}
biome_ecosystem = {}

relation_biome_ecozone = set()
relation_ecozone_ecosystem = set()

for index, row in biome.iterrows():
    biome_ = row['biome']
    code = row['code']

    level = len(code.split('.'))

    if level == 1:
        #biome_biome.add((biome_, code))
        biome_biome[code] = biome_
    elif level == 2:
        #biome_ecozone.add((biome_, code))
        biome_ecozone[code] = biome_
        father_code = code.split('.')[0]
        relation_biome_ecozone.add((father_code, code))
    elif level == 3:
        #biome_ecosystem.add((biome_, code))
        biome_ecosystem[code] = biome_
        father_code = code.split('.')[0] + '.' + code.split('.')[1]
        relation_ecozone_ecosystem.add((father_code, code))

with open(os.path.join(for_neo4j_folder, 'node_biome_biome.tsv'), 'w') as file:
    file.write('name\tcode\n')
    for code in biome_biome:
        file.write(f'{biome_biome[code]}\t{code}\n')

with open(os.path.join(for_neo4j_folder, 'node_biome_ecozone.tsv'), 'w') as file:
    file.write('name\tcode\n')
    for code in biome_ecozone:
        file.write(f'{biome_ecozone[code]}\t{code}\n')

with open(os.path.join(for_neo4j_folder, 'node_biome_ecosystem.tsv'), 'w') as file:
    file.write('name\tcode\n')
    for code in biome_ecosystem:
        file.write(f'{biome_ecosystem[code]}\t{code}\n')

with open(os.path.join(for_neo4j_folder, 'relation_biome_ecozone.tsv'), 'w') as file:
    file.write('from\tto\n')
    for father_code, code in relation_biome_ecozone:
        file.write(f'{code}\t{father_code}\n')

with open(os.path.join(for_neo4j_folder, 'relation_biome_ecozone_ecosystem.tsv'), 'w') as file:
    file.write('from\tto\n')
    for father_code, code in relation_ecozone_ecosystem:
        file.write(f'{code}\t{father_code}\n')

In [32]:
countries_region = set()
countries_subregion = set()
countries_countries = {}

countries_countries["WLD"] = "World"

relation_region_subregion = set()
relation_subregion_country = set()

relation_region_subregion.add(("World", "World"))
relation_subregion_country.add(("World", "WLD"))

for index, row in countries.iterrows():
    code = row['alpha-3']
    name = row['name']
    region = row['region']
    sub_region = row['sub-region']

    countries_region.add(region)
    countries_subregion.add(sub_region)
    countries_countries[code] = name

    relation_region_subregion.add((region, sub_region))
    relation_subregion_country.add((sub_region, code))

with open(os.path.join(for_neo4j_folder, 'node_countries_region.tsv'), 'w') as file:
    file.write('name\n')
    for region in countries_region:
        file.write(f'{region}\n')

with open(os.path.join(for_neo4j_folder, 'node_countries_subregion.tsv'), 'w') as file:
    file.write('name\n')
    for sub_region in countries_subregion:
        file.write(f'{sub_region}\n')

with open(os.path.join(for_neo4j_folder, 'node_countries_countries.tsv'), 'w') as file:
    file.write('code\tname\n')
    for code in countries_countries:
        file.write(f'{code}\t{countries_countries[code]}\n')

with open(os.path.join(for_neo4j_folder, 'relation_region_subregion.tsv'), 'w') as file:
    file.write('from\tto\n')
    for region, sub_region in relation_region_subregion:
        file.write(f'{sub_region}\t{region}\n')

with open(os.path.join(for_neo4j_folder, 'relation_subregion_country.tsv'), 'w') as file:
    file.write('from\tto\n')
    for sub_region, code in relation_subregion_country:
        file.write(f'{code}\t{sub_region}\n')


In [33]:
data_df.head()

,ValueId,StudyId,ESVD2.0_Biome,ESVD2.0_Biome_Codes,ESVD2.0_Biome_1,ESVD2.0_Biome_2,ESVD2.0_Biome_3,ESVD2.0_Biome_4,ESVD2.0_Biome_5,ESVD2.0_Biome_6,...,Type Of Beneficiary,Int$ Per Hectare Per Year,Authors,Title,Reference,Year_Pub,Notes,Secondary Data Sources,Reviewed,inclExcl
0,12410006,1241,Urban and industrial areas,15,Urban and industrial areas,NaN,NaN,NaN,NaN,NaN,...,residents,NaN,Bronnmann et al,The Value of Naturalness of Urban Green Spaces...,"Bronnmann, J., Liebelt, V., Marder, F., Meya, ...",2020,The study does not provide the area size. Fill...,NaN,Yes,NaN
1,12410005,1241,Urban and industrial areas,15,Urban and industrial areas,NaN,NaN,NaN,NaN,NaN,...,residents,NaN,Bronnmann et al,The Value of Naturalness of Urban Green Spaces...,"Bronnmann, J., Liebelt, V., Marder, F., Meya, ...",2020,The study does not provide the area size. Fill...,NaN,Yes,NaN
2,12410004,1241,Urban and industrial areas,15,Urban and industrial areas,NaN,NaN,NaN,NaN,NaN,...,residents,NaN,Bronnmann et al,The Value of Naturalness of Urban Green Spaces...,"Bronnmann, J., Liebelt, V., Marder, F., Meya, ...",2020,The study does not provide the area size. Fill...,NaN,Yes,NaN
3,12410003,1241,Urban and industrial areas,15,Urban and industrial areas,NaN,NaN,NaN,NaN,NaN,...,residents,NaN,Bronnmann et al,The Value of Naturalness of Urban Green Spaces...,"Bronnmann, J., Liebelt, V., Marder, F., Meya, ...",2020,The study does not provide the area size. Fill...,NaN,Yes,NaN
4,12410002,1241,Urban and industrial areas,15,Urban and industrial areas,NaN,NaN,NaN,NaN,NaN,...,residents,NaN,Bronnmann et al,The Value of Naturalness of Urban Green Spaces...,"Bronnmann, J., Liebelt, V., Marder, F., Meya, ...",2020,The study does not provide the area size. Fill...,NaN,Yes,NaN


In [34]:
data_df.fillna({'Protection Status': 'No protection'}, inplace=True)
data_df.fillna({'Valuation Methods': 'OT'}, inplace=True)

In [35]:
values = {}
seea = set()


for index, row in data_df.iterrows():
    value_id = row['ValueId']
    study_id = row['StudyId']
    
    
    values[value_id] = {"study_id": study_id, "biome": set(), "ecosystem": set(), "ecozone": set(),
                         "teeb_service": set(), "teeb_subservice": set(), "cices": set(), "seea": set(), "countries": set()}
    values[value_id]["study_location"] = row['Study Location']
    values[value_id]["protection_status"] = row['Protection Status']
    values[value_id]["latitude"] = row['Latitude']
    values[value_id]["longitude"] = row['Longitude']

    #print (row['Valuation Methods'])
    values[value_id]["valuation_methods"] = "|".join([x.strip() for x in str(row['Valuation Methods']).split(";") if str(row['Valuation Methods']) != "nan"])


    values[value_id]["beneficiary_unit"] = str(row['Beneficiary Unit']).lower()

    if values[value_id]["beneficiary_unit"].endswith('s'):
        values[value_id]["beneficiary_unit"] = values[value_id]["beneficiary_unit"][:-1]

    if values[value_id]["beneficiary_unit"] == "individual":
        values[value_id]["beneficiary_unit"] = "person"

    values[value_id]["value_year"] = row['Value Year']
    values[value_id]["int__per_hectare_per_year"] = row['Int$ Per Hectare Per Year']

    if values[value_id]["int__per_hectare_per_year"] > 1000000 or values[value_id]["int__per_hectare_per_year"] < 1:
        values[value_id]["int__per_hectare_per_year"] = 0

    values[value_id]["title"] = row['Title']
    values[value_id]["reference"] = row['Reference']
    values[value_id]["year_pub"] = row['Year_Pub']
    values[value_id]["site_area_in_hectares"] = row['Site Area In Hectares']
    values[value_id]["site_length_km"] = row['Site Length Km']

    ecosystem_codes = [x.strip() for x in str(row['ESVD2.0_Ecosystem_Codes']).split(';') if str(row['ESVD2.0_Ecosystem_Codes']) != "nan"]
    ecozone_codes = [x.strip() for x in str(row['ESVD2.0_Ecozone_Codes']).split(';') if str(row['ESVD2.0_Ecozone_Codes']) != "nan"]
    biome_codes = [x.strip() for x in str(row['ESVD2.0_Biome_Codes']).split(';') if str(row['ESVD2.0_Biome_Codes']) != "nan"]

    if len(ecosystem_codes) > 0:
        for ecosystem_code in ecosystem_codes:
            #relation_ecosystem_value.add((ecosystem_code, value_id))
            values[value_id]["ecosystem"].add(ecosystem_code)
    elif len(ecozone_codes) > 0:
        for ecozone_code in ecozone_codes:
            values[value_id]["ecozone"].add(ecozone_code)
            #relation_ecozone_value.add((ecozone_code, value_id))
    else:
        for biome_code in biome_codes:
            values[value_id]["biome"].add(biome_code)
            #relation_biome_value.add((biome_code, value_id))

    
    teeb_subes_codes = [x.strip() for x in str(row['TEEB_SubES']).split(';') if str(row['TEEB_SubES']) != "nan"]
    teeb_es_codes = [x.strip() for x in str(row['TEEB_ES']).split(';') if str(row['TEEB_ES']) != "nan"]  

    if len(teeb_subes_codes) > 0:
        for teeb_subes_code in teeb_subes_codes:
            values[value_id]["teeb_subservice"].add(teeb_subes_code)
            #relation_teeb_subes_value.add((teeb_subes_code, value_id))
    else:
        for teeb_es_code in teeb_es_codes:
            values[value_id]["teeb_service"].add(teeb_es_code)
            #relation_teeb_es_value.add((teeb_es_code, value_id))

    cices_codes = [x.strip() for x in str(row['CICES']).split(';') if str(row['CICES']) != "nan"]

    if len(cices_codes) > 0:
        for cices_code in cices_codes:
            values[value_id]["cices"].add(cices_code)
            #relation_cices_value.add((cices_code, value_id))

    seea_codes = [x.strip() for x in str(row['SEEA']).split(';') if str(row['SEEA']) != "nan"]

    if len(seea_codes) > 0:
        for seea_code in seea_codes:
            seea.add(seea_code)
            values[value_id]["seea"].add(seea_code)
            #relation_seea_value.add((seea_code, value_id))

    countries_codes = [x.strip() for x in str(row['Country_Codes']).split(';') if str(row['Country_Codes']) != "nan"]

    if len(countries_codes) > 0:
        for countries_code in countries_codes:
            values[value_id]["countries"].add(countries_code)
            #relation_countries_value.add((countries_code, value_id))

with open(os.path.join(for_neo4j_folder, 'node_seea.tsv'), 'w') as file:
    file.write('name\n')
    for code in seea:
        file.write(f'{code}\n')

with open(os.path.join(for_neo4j_folder, 'node_values.tsv'), 'w') as file:
    file.write('name\tstudy_id\tstudy_location\tprotection_status\tlatitude\tlongitude\tvaluation_methods\tbeneficiary_unit\tvalue_year\tint__per_hectare_per_year\ttitle\treference\tyear_pub\tsite_area_in_hectares\tsite_length_km\n')
    for value_id in values:
        file.write(f'{value_id}\t{values[value_id]["study_id"]}\t{values[value_id]["study_location"]}\t{values[value_id]["protection_status"]}\t{values[value_id]["latitude"]}\t{values[value_id]["longitude"]}\t{values[value_id]["valuation_methods"]}\t{values[value_id]["beneficiary_unit"]}\t{values[value_id]["value_year"]}\t{values[value_id]["int__per_hectare_per_year"]}\t{values[value_id]["title"]}\t{values[value_id]["reference"]}\t{values[value_id]["year_pub"]}\t{values[value_id]["site_area_in_hectares"]}\t{values[value_id]["site_length_km"]}\n')

with open(os.path.join(for_neo4j_folder, 'relation_biome_value.tsv'), 'w') as file:
    file.write('from\tto\n')
    for value_id in values:
        for biome in values[value_id]["biome"]:
            file.write(f'{value_id}\t{biome}\n')

with open(os.path.join(for_neo4j_folder, 'relation_biome_ecosystem_value.tsv'), 'w') as file:
    file.write('from\tto\n')
    for value_id in values:
        for ecosystem in values[value_id]["ecosystem"]:
            file.write(f'{value_id}\t{ecosystem}\n')

with open(os.path.join(for_neo4j_folder, 'relation_biome_ecozone_value.tsv'), 'w') as file:
    file.write('from\tto\n')
    for value_id in values:
        for ecozone in values[value_id]["ecozone"]:
            file.write(f'{value_id}\t{ecozone}\n')

with open(os.path.join(for_neo4j_folder, 'relation_teeb_service_value.tsv'), 'w') as file:
    file.write('from\tto\n')
    for value_id in values:
        for teeb in values[value_id]["teeb_service"]:
            file.write(f'{value_id}\t{teeb}\n')

with open(os.path.join(for_neo4j_folder, 'relation_teeb_subservice_value.tsv'), 'w') as file:
    file.write('from\tto\n')
    for value_id in values:
        for teeb in values[value_id]["teeb_subservice"]:
            file.write(f'{value_id}\t{teeb}\n')

with open(os.path.join(for_neo4j_folder, 'relation_cices_class_value.tsv'), 'w') as file:
    file.write('from\tto\n')
    for value_id in values:
        for cices in values[value_id]["cices"]:
            file.write(f'{value_id}\t{cices}\n')

with open(os.path.join(for_neo4j_folder, 'relation_seea_value.tsv'), 'w') as file:
    file.write('from\tto\n')
    for value_id in values:
        for seea in values[value_id]["seea"]:
            file.write(f'{value_id}\t{seea}\n')

with open(os.path.join(for_neo4j_folder, 'relation_countries_value.tsv'), 'w') as file:
    file.write('from\tto\n')
    for value_id in values:
        for countries in values[value_id]["countries"]:
            file.write(f'{value_id}\t{countries}\n')
        

In [36]:


url = "bolt://localhost:7687"
driver = GraphDatabase.driver(url, auth=(PARAM["neo4j_username"], PARAM["neo4j_password"]))


with driver.session() as session:
    # Drop constraints / indices
    for constraint in session.run("SHOW CONSTRAINT"):

        #print (constraint["name"])
        session.run("DROP CONSTRAINT " + constraint["name"] + ";")

    # delete all nodes    
    session.run("MATCH (n) DETACH DELETE n")



In [37]:
src = "for_neo4j"

home = str(Path.home())

#file:///home/dgg32/.config/Neo4j%20Desktop/Application/relate-data/dbmss

dst = os.path.join(home, ".config", "Neo4j Desktop", "Application", "relate-data", "dbmss", PARAM["neo4j_project_id"], "import")


for src_dir, dirs, files in os.walk(src):
    for file_ in files:
        src_file = os.path.join(src_dir, file_)
        dst_file = os.path.join(dst, file_)
        shutil.copy(src_file, dst_file)